Niezbędne biblioteki

In [33]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import confusion_matrix

# Czyszczenie i przygotowanie danych do modelowania

In [34]:
# Do wywietlania wszystkich wartoci z DataFrame
pd.set_option('display.max_columns', None)
pd.set_option("display.max_rows", None)

In [35]:
#wczytujemy dane i wywietlamy informacje o rozmiarze DataFrame
loan_data = pd.read_csv("E:/Materialy_future/Loan_data.csv", low_memory = False)
loan_data.shape

(42536, 151)

In [36]:
loan_data.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,1077501,NaN,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-2011,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/22/11 > I need to upgra...,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-1985,735.0,739.0,1.0,NaN,NaN,3.0,0.0,13648.0,83.7%,9.0,f,0.0,0.0,5863.155187,5833.84,5000.00,863.16,0.00,0.0,0.00,Jan-2015,171.62,NaN,Aug-2018,739.0,735.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,1077430,NaN,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/22/11 > I plan to use t...,car,bike,309xx,GA,1.00,0.0,Apr-1999,740.0,744.0,5.0,NaN,NaN,3.0,0.0,1687.0,9.4%,4.0,f,0.0,0.0,1014.530000,1014.53,456.46,435.17,0.00,122.9,1.11,Apr-2013,119.66,NaN,Oct-2016,499.0,0.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [37]:
#Usuwamy kolumny zawierajce tylko puste pola (NaN)
loan_data.dropna(axis = 1, how = 'all', inplace = True)
# Usuwam wiersze z wartociami NaN dla kluczorej kolumny
loan_data.dropna(subset = ['loan_amnt'], inplace = True)
loan_data.shape

(42535, 69)

In [38]:
#Zostawiamy dane zawierające w kolumnie 'loan_status' tylko dwie unikalne wartości
loan_data = loan_data[(loan_data.loan_status=='Charged Off')|
          (loan_data['loan_status']=='Fully Paid')]
loan_data.shape

(39786, 69)

In [39]:
#Usuwanie kolumn zawierających ponad 70% wartości NaN
threshold = 0.7
loan_data = loan_data[loan_data.columns[loan_data.isnull().mean() < threshold]]
loan_data.shape

(39786, 61)

In [40]:
#Usunięcie kolumn tylko z jedną unikalną wartością
loan_data = loan_data[loan_data.columns[loan_data.nunique() > 1]]
loan_data.shape

(39786, 48)

In [41]:
#usuwam kolumny zawierające zbędne informacje 
loan_data.drop(['id', 
                'emp_title',
                'addr_state',
                'url', 
                'desc', 
                'purpose', 
                'title', 
                'collection_recovery_fee'], axis=1, inplace = True) 

In [42]:
#usuwam kolumnę 'fico_range_high', bo 'fico_range_low' jest taka sama pomniejszona o 4
loan_data.drop(['fico_range_high'], axis=1, inplace = True)

In [43]:
#usuwam kolumny zawierające dane z przyszłości
loan_data.drop(['last_pymnt_d', 
                'last_pymnt_amnt', 
                'last_credit_pull_d', 
                'last_fico_range_high', 
                'last_fico_range_low'], axis=1, inplace = True)

Do modelowania konieczna jest zmiana danych tekstowych (object) na numeryczne (int/float)

In [56]:
loan_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39786 entries, 0 to 39785
Data columns (total 33 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   loan_amnt               39786 non-null  float64       
 1   funded_amnt             39786 non-null  float64       
 2   funded_amnt_inv         39786 non-null  float64       
 3   term                    39786 non-null  int64         
 4   int_rate                39786 non-null  float64       
 5   installment             39786 non-null  float64       
 6   emp_length              39786 non-null  float64       
 7   home_ownership          39786 non-null  object        
 8   annual_inc              39786 non-null  float64       
 9   verification_status     39786 non-null  int64         
 10  issue_d                 39786 non-null  datetime64[ns]
 11  loan_status             39786 non-null  int64         
 12  zip_code                39786 non-null  int32 

In [45]:
#Usuwanie częsci tekstowej
loan_data['term'].replace({' 36 months':36, 
                           ' 60 months':60}, inplace = True)
loan_data['zip_code'] = loan_data['zip_code'].str.rstrip('xx').astype('int')

In [46]:
#Przekształcenie procentów
loan_data['int_rate'] = loan_data['int_rate'].str.rstrip('%').astype('float') / 100.0
loan_data['revol_util'] = loan_data['revol_util'].str.rstrip('%').astype('float')/100

In [47]:
#Dane kategoryczne
loan_data['grade'].replace({'A':1, 
                            'B':2, 
                            'C':3, 
                            'D':4, 
                            'E':5, 
                            'F':6, 
                            'G':7}, inplace = True)
loan_data['emp_length'].replace({'10+ years':10, 
                            '1 year':1, 
                            '3 years':3, 
                            '8 years':8, 
                            '9 years':9, 
                            '4 years':4, 
                            '5 years':5,
                            '6 years':6,
                            '2 years':2,
                            '7 years':7,
                            '< 1 year':0}, inplace = True)
loan_data['verification_status'].replace({'Verified':1, 
                                          'Source Verified':1, 
                                          'Not Verified':0}, inplace = True)
loan_data['loan_status'].replace({'Fully Paid':1, 
                                  'Charged Off':0}, inplace = True)
loan_data['debt_settlement_flag'].replace({'Y':1, 
                                           'N':0}, inplace = True)

In [48]:
#Uzupełnienie kolumn zawierających puste pola wartością 0
loan_data[["emp_length", 
           "mths_since_last_delinq", 
           "pub_rec_bankruptcies",
          "revol_util"]] = loan_data[["emp_length", 
                                                 "mths_since_last_delinq", 
                                                 "pub_rec_bankruptcies",
                                     "revol_util"]].fillna(0)

# loan_data["mths_since_last_delinq"] = loan_data["mths_since_last_delinq"].fillna(0)
# loan_data["pub_rec_bankruptcies"] = loan_data["pub_rec_bankruptcies"].fillna(0)

In [49]:
#Zmiana na format czasowy
loan_data['issue_d'] = pd.to_datetime(loan_data['issue_d'], format='%b-%Y')
loan_data['earliest_cr_line'] = pd.to_datetime(loan_data['earliest_cr_line'], format='%b-%Y')

In [50]:
#Przekształcamy ocenę z kategorycznej do numerycznej kolumny 
loan_data['sub_grade'] = loan_data['sub_grade'].str[1:]
loan_data['sub_grade'] = pd.to_numeric(loan_data['sub_grade'])
#Ocena numeryczna
loan_data['numeric_grade'] = loan_data['grade']+ loan_data['sub_grade']/10
loan_data.drop(['sub_grade', 
                'grade'], axis=1, inplace = True)

In [57]:
#Zmieniamy kolumnę 'home_ownership' na kategoryczną za pomocą funkcji get_dummies
loan_data_dumm = pd.get_dummies(loan_data, columns=['home_ownership'])

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,emp_length,annual_inc,verification_status,issue_d,loan_status,zip_code,dti,delinq_2yrs,earliest_cr_line,fico_range_low,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,pub_rec_bankruptcies,debt_settlement_flag,numeric_grade,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT
0,5000.0,5000.0,4975.0,36,0.1065,162.87,10.0,24000.0,1,2011-12-01,1,860,27.65,0.0,1985-01-01,735.0,1.0,0.0,3.0,0.0,13648.0,0.837,9.0,5863.155187,5833.84,5000.00,863.16,0.00,0.0,0.0,0,2.2,0,0,0,0,1
1,2500.0,2500.0,2500.0,60,0.1527,59.83,0.0,30000.0,1,2011-12-01,0,309,1.00,0.0,1999-04-01,740.0,5.0,0.0,3.0,0.0,1687.0,0.094,4.0,1014.530000,1014.53,456.46,435.17,0.00,122.9,0.0,0,3.4,0,0,0,0,1
2,2400.0,2400.0,2400.0,36,0.1596,84.33,10.0,12252.0,0,2011-12-01,1,606,8.72,0.0,2001-11-01,735.0,2.0,0.0,2.0,0.0,2956.0,0.985,10.0,3005.666844,3005.67,2400.00,605.67,0.00,0.0,0.0,0,3.5,0,0,0,0,1
3,10000.0,10000.0,10000.0,36,0.1349,339.31,10.0,49200.0,1,2011-12-01,1,917,20.00,0.0,1996-02-01,690.0,1.0,35.0,10.0,0.0,5598.0,0.210,37.0,12231.890000,12231.89,10000.00,2214.92,16.97,0.0,0.0,0,3.1,0,0,0,0,1
4,3000.0,3000.0,3000.0,60,0.1269,67.79,1.0,80000.0,1,2011-12-01,1,972,17.94,0.0,1996-01-01,695.0,0.0,38.0,15.0,0.0,27783.0,0.539,38.0,4066.908161,4066.91,3000.00,1066.91,0.00,0.0,0.0,0,2.5,0,0,0,0,1


In [63]:
loan_data_dumm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39786 entries, 0 to 39785
Data columns (total 37 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   loan_amnt                39786 non-null  float64       
 1   funded_amnt              39786 non-null  float64       
 2   funded_amnt_inv          39786 non-null  float64       
 3   term                     39786 non-null  int64         
 4   int_rate                 39786 non-null  float64       
 5   installment              39786 non-null  float64       
 6   emp_length               39786 non-null  float64       
 7   annual_inc               39786 non-null  float64       
 8   verification_status      39786 non-null  int64         
 9   issue_d                  39786 non-null  datetime64[ns]
 10  loan_status              39786 non-null  int64         
 11  zip_code                 39786 non-null  int32         
 12  dti                      39786 n

In [62]:
loan_data['revol_util'].unique()

array([8.370e-01, 9.400e-02, 9.850e-01, ..., 4.963e-01, 4.000e-04,
       7.280e-02])